<a href="https://colab.research.google.com/github/ggaoshen/graph_wavenet/blob/sg-experiment/CS224W_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CS224W - Final Project**

Description goes here.

# Device
You might need to use GPU for this Colab.

Please click `Runtime` and then `Change runtime type`. Then set the `hardware accelerator` to **GPU**.

# Installation

In [1]:
# Install torch geometric
import os
import torch
if 'IS_GRADESCOPE_ENV' not in os.environ:
  torch_version = str(torch.__version__)
  scatter_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
  sparse_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
  !pip install torch-scatter -f $scatter_src
  !pip install torch-sparse -f $sparse_src
  !pip install torch-geometric
  !pip install -q git+https://github.com/snap-stanford/deepsnap.git
  !pip install -U -q PyDrive

Looking in links: https://pytorch-geometric.com/whl/torch-2.1.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 41.0 MB/s eta 0:00:00
Looking in links: https://pytorch-geometric.com/whl/torch-2.1.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
  !nvcc --version
  !python -c "import torch; print(torch.version.cuda)"

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
11.8


In [3]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
  import torch
  print(torch.__version__)
  import torch_geometric
  print(torch_geometric.__version__)

2.1.0+cu118
2.4.0


In [4]:
# Import GraphWaveNet module
!git clone -b mmajid/tgnn-improvements https://github.com/ggaoshen/graph_wavenet.git # NOTE: choose the right branch
%load graph_wavenet/src/graphwavenet.py
import sys
sys.path.append('graph_wavenet/src/')
from graphwavenet import GraphWaveNet
from util import masked_rmse, masked_mse, masked_mae, masked_mape, metric, temporal_dataset_split

Cloning into 'graph_wavenet'...
remote: Enumerating objects: 159, done.
remote: Counting objects: 100% (159/159), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 159 (delta 88), reused 72 (delta 26), pack-reused 0
Receiving objects: 100% (159/159), 108.04 KiB | 2.84 MiB/s, done.
Resolving deltas: 100% (88/88), done.


In [5]:
!pip install -U torch-geometric-temporal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 884.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 30.1 MB/s eta 0:00:00
  Created wheel for torch-geometric-temporal: filename=torch_geometric_temporal-0.54.0-py3-none-any.whl size=86722 sha256=2f53a1cdcff26cf23c4c5e0526bc4bd953ab373e0d6610ff0067d6ff51af891a
  Stored in directory: /root/.cache/pip/wheels/9e/9b/b6/e15256e053f0cb49b1084a67a709db909d418386a231f0722c
Successfully built torch-geometric-temporal
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires py

In [6]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context



# Temporal Datasets
# from torch_geometric_temporal.dataset import METRLADatasetLoader
# loader = METRLADatasetLoader()
# dataset = loader.get_dataset(num_timesteps_in=12, num_timesteps_out=12)
# iterator = iter(dataset)
# print("METRLADatasetLoader", next(iterator))
# print("METRLADatasetLoader", next(iterator))


from torch_geometric_temporal.dataset import ChickenpoxDatasetLoader
dataset = ChickenpoxDatasetLoader().get_dataset(lags=8) # consistent with chickenpox paper
iterator = iter(dataset)
print("ChickenpoxDatasetLoader", next(iterator))
print("ChickenpoxDatasetLoader", next(iterator))


# from torch_geometric_temporal.dataset import WikiMathsDatasetLoader
# dataset = WikiMathsDatasetLoader().get_dataset(lags=14)
# iterator = iter(dataset)
# print("WikiMathsDatasetLoader", next(iterator))
# print("WikiMathsDatasetLoader", next(iterator))


# from torch_geometric_temporal.dataset import TwitterTennisDatasetLoader
# dataset = TwitterTennisDatasetLoader().get_dataset()
# print("TwitterTennisDatasetLoader", next(iter(dataset)))


# Graph summaries
# METRLADatasetLoader Data(x=[207, 2, 12], edge_index=[2, 1722], edge_attr=[1722], y=[207, 12])
# ChickenpoxDatasetLoader Data(x=[20, 4], edge_index=[2, 102], edge_attr=[102], y=[20])
# WikiMathsDatasetLoader Data(x=[1068, 14], edge_index=[2, 27079], edge_attr=[27079], y=[1068])

ChickenpoxDatasetLoader Data(x=[20, 8], edge_index=[2, 102], edge_attr=[102], y=[20])
ChickenpoxDatasetLoader Data(x=[20, 8], edge_index=[2, 102], edge_attr=[102], y=[20])


In [ ]:
# Run Chickenpox Dataset

# from torch_geometric_temporal.signal import temporal_signal_split

import torch
import torch.nn.functional as F
import numpy as np

# training
from tqdm import tqdm
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache() # clear cuda cache

# sample training loop
train_split, validation_split = 0.6, 0.2
train_dataset, val_dataset, test_dataset = temporal_dataset_split(dataset, train_split=train_split, validation_split=validation_split)
train_size = int(train_split * dataset.snapshot_count)
val_size = int(validation_split * dataset.snapshot_count)
val_offset, test_offset = train_size, train_size + val_size # starting index for val and test sets

in_dim = dataset[0].num_node_features # 8 treat lagged inputs as node features
out_dim = 1 # output size
num_nodes = dataset[0].num_nodes # 1068
timesteps_to_predict = 10 # 1
epochs = 200
lrate = 0.0001
wdecay = 0.001
save_path = "store/checkpoint"

model = GraphWaveNet(
    num_nodes=num_nodes,
    in_channels=in_dim,
    out_channels=out_dim,
    out_timesteps=timesteps_to_predict,
).to(device)

if not os.path.exists(save_path):
    os.makedirs(save_path)

print("start training...", flush=True)
his_loss = []
val_time = []
train_time = []
best_epoch = 0

optimizer = torch.optim.Adam(model.parameters(), lr=lrate, weight_decay=wdecay)

model.train()

# def get_temporal_y_label(y_1d, tag="train"):
#     """
#     y_1d: y label at timestep t,
#     Return: y label over lookback window [t + 1 - timesteps_to_predict, t]
#     """
#     if timesteps_to_predict == 1:
#         return snapshot.y.to(device)
#     else:
#         y_train = torch.stack([dataset[t].y for t in range(i+1-timesteps_to_predict, i+1)], dim=0).to(device)

for epoch in tqdm(range(epochs)):
    train_loss = []
    t1 = time.time()

    for i, snapshot in enumerate(train_dataset):
        x_train = snapshot.x.reshape(-1, num_nodes, in_dim).to(device)
        # print("x_train size:", x_train.shape)
        y_train = snapshot.y.to(device)
        pred = model(x_train, snapshot.edge_index, snapshot.edge_attr).squeeze()

        # print(pred.size())
        # print(y_train.size())
        # print(y_train[-1])
        # print(snapshot.y)
        loss = masked_mse(pred, y_train, 0.0) # mean squared error for loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        train_loss.append(loss.item())

        # if i % 100 == 0:
        #     log = 'Iter: {:05d}, Train Loss: {:.4f}'
        #     print(
        #         log.format(
        #             i,
        #             train_loss[-1]), flush=True)


    t2 = time.time()
    train_time.append(t2 - t1)
    valid_loss = []

    s1 = time.time()
    for i, snapshot in enumerate(val_dataset):
        x_val = snapshot.x.reshape(-1, num_nodes, in_dim).to(device)
        y_val = snapshot.y.to(device)
        pred = model(x_val, snapshot.edge_index, snapshot.edge_attr).squeeze()

        loss = masked_mse(pred, y_val, 0.0) # mean squared error for loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        valid_loss.append(loss.item())

    s2 = time.time()
    log = 'Epoch: {:05d}, Inference Time: {:.4f} secs'
    print(log.format(epoch, (s2 - s1)))
    val_time.append(s2 - s1)
    mtrain_loss = np.mean(train_loss)
    mvalid_loss = np.mean(valid_loss)
    his_loss.append(mvalid_loss)

    if np.argmin(his_loss) == len(his_loss) - 1:
        best_epoch = epoch

    log = 'Epoch: {:03d}, Train Loss: {:.4f},' + \
        'Valid Loss: {:.4f},' + \
        'Training Time: {:.4f}/epoch'
    print(
        log.format(epoch, mtrain_loss, mvalid_loss, (t2 - t1)), flush=True)

print("Average Training Time: {:.4f} secs/epoch".format(
    np.mean(train_time)))
print("Average Inference Time: {:.4f} secs".format(np.mean(val_time)))

pd.DataFrame({"train_loss": train_loss, "val_loss": valid_loss}).to_csv("training_curve.csv") # store training and testing loss

# eval
model.eval()
loss = 0
for i, snapshot in enumerate(test_dataset):
    x_test = snapshot.x.reshape(-1, num_nodes, in_dim).to(device)
    y_test = snapshot.y.to(device)

    with torch.no_grad():
        pred = model(x_test, snapshot.edge_index, snapshot.edge_attr)
    loss += masked_mse(pred, y_test, 0.0) # mean absolute error as loss

loss = loss / (i+1)
loss = loss.item()
print("MSE: {:.4f}".format(loss))


start training...


  0%|          | 0/200 [00:00<?, ?it/s]

Epoch: 00000, Inference Time: 5.2831 secs
Epoch: 000, Train Loss: 0.8733,Valid Loss: 0.6162,Training Time: 14.0189/epoch


  0%|          | 1/200 [00:19<1:04:01, 19.30s/it]

Epoch: 00001, Inference Time: 4.3150 secs
Epoch: 001, Train Loss: 0.7433,Valid Loss: 0.5889,Training Time: 14.7616/epoch


...

  4%|▎         | 7/200 [02:14<1:00:53, 18.93s/it]

Epoch: 00007, Inference Time: 5.0008 secs
Epoch: 007, Train Loss: 0.6549,Valid Loss: 0.5331,Training Time: 13.7769/epoch


  4%|▍         | 8/200 [02:33<1:00:25, 18.88s/it]

Epoch: 00008, Inference Time: 4.3093 secs
Epoch: 008, Train Loss: 0.6442,Valid Loss: 0.5293,Training Time: 13.5975/epoch


  4%|▍         | 9/200 [02:51<59:08, 18.58s/it]  

In [ ]:
!nvidia-smi

In [ ]:
# Reset GPU RAM if GPU out of memory

# !pip install numba

# from numba import cuda

# cuda.select_device(0) # choosing second GPU

# cuda.close()